In [1]:
import pandas as pd
import numpy as np
from io import StringIO
import re
from scipy.optimize import fsolve
from astropy import constants as const

In [2]:
import tools
%load_ext autoreload
%autoreload 2

First, we will generate our data (radial velocity values) using a simulation!

The radial velocity of a star in a binary system is given by:
$$v(t) = \kappa[cos(f+\omega)+e \cos\omega + v_0$$
where
$$\kappa =\dfrac{(2\pi G)^{1/3}m\sin I}{T^{1/3}(M+m)^{2/3}\sqrt{1-e^2}}$$

$$tan(f/2)= \sqrt{\dfrac{1+e}{1-e}}\tan(u/2)$$

$$u-e\sin u = \dfrac{2\pi}{T}(t-\tau)$$

#### insert descr. of parameters!!!!!!!!! ####

For each parameter (like eccentricity) we have a range of possible values. These are as follows: 

########## insert ###########

Now, for each parameter, we'll randomly select values from a uniform distribution of their possible values.



In [65]:
def run_sim(sim_duration=3e8, num_data_points=100):
    """ (int, int) -> (pd.DataFrame)
    Arguments:
        sim_duration: (int) time elapsed during simulation
        num_data_points: (int) number of data points in simulation 
    Returns a pandas DataFrame with time and radial velocity
    """
    m = np.random.uniform(0, 1) # in solar masses
    M = np.random.uniform(0.072, 150) # in solar masses
    e = np.random.uniform(0, 1)
    I = np.random.uniform(-np.pi, np.pi)
    omega = np.random.uniform(0, np.pi/2)
    T = np.random.uniform(3282.3503, 3.46896e13) # in seconds
    tau = np.random.uniform(3282.3503, 3.46896e13) # in seconds
    v_0 = np.random.uniform(-1000, 1000) # in m/s
    
    t = np.linspace(0, sim_duration, num_data_points)
    
    radial_velocities = tools.radial_velocity(t, m, M, T, I, e, v_0, omega, tau)
    # adding random Gaussian noise
    radial_velocities += np.random.normal(20, 10, len(t))
    
    data = {'Time':t, 'Radial Velocity':radial_velocities}
    df = pd.DataFrame(data)
    return df


In [77]:
class BinarySystem:
    """Represents a Binary System
    """
    def __init__(self, time=None, radial_velocity=None):
        # if no input radial velocity
        # will generate with simulation
        if radial_velocity == None:
            # generating values for parameters
            self.m = np.random.uniform(0, 1) # in solar masses
            self.M = np.random.uniform(0.072, 150) # in solar masses
            self.e = np.random.uniform(0, 1)
            self.I = np.random.uniform(-np.pi, np.pi)
            self.omega = np.random.uniform(0, np.pi/2)
            self.T = np.random.uniform(3282.3503, 3.46896e13) # in seconds
            self.tau = np.random.uniform(3282.3503, 3.46896e13) # in seconds
            self.v_0 = np.random.uniform(-1000, 1000) # in m/s
            
            # generating radial velocity data from 
            t = np.linspace(0, 3e8, 100)
            self.time = t
            radial_velocities = tools.radial_velocity(t, self.m, self.M, self.T, 
                                                      self.I, self.e, self.v_0, self.omega, self.tau)
            # adding random Gaussian noise
        radial_velocities += np.random.normal(20, 10, len(t))
            self.radial_velocity = radial_velocities
        # if there is a radial velocity input, use input   
        else:
            self.time = time 
            self.radial_velocity = radial_velocity


In [ ]:
Finally, we will add random Gaussian noise to each simulated radial velocity value. 

Now, we will try to recover these parameter values using an MCMC!